In [1]:
%load_ext autoreload
%autoreload 2
%env PYTORCH_ENABLE_MPS_FALLBACK=1
import os
os.chdir("/home/kubach/project_sync/star_analysis")
print(os.getcwd())

env: PYTORCH_ENABLE_MPS_FALLBACK=1


In [2]:
from star_analysis.model.types import ModelTypes
from star_analysis.runner.sdss_runner import SdssRunner
from star_analysis.data.augmentations import Augmentations
from star_analysis.runner.sdss_runner import SdssRunConfig, SdssModelConfig
from star_analysis.model.neural_networks.losses.types import LossType
from star_analysis.runner.run import Run

In [3]:
runner = SdssRunner(project_name="sdss-hparams")

In [4]:
from functools import partial
import copy
from star_analysis.runner.run import OptunaTuneTrainerConfig
from optuna import Trial

def objective(trial: Trial, runner: SdssRunner, default_run_config: SdssRunConfig, trainer_config: OptunaTuneTrainerConfig):
    lr = trial.suggest_loguniform("learning_rate", 1e-6, 1e-3)
    arch = trial.suggest_categorical("architecture", [ModelTypes.UNET, ModelTypes.DLV3])

    run_config = copy.deepcopy(default_run_config)
    if arch == ModelTypes.UNET:
        run_config.model_type = ModelTypes.UNET
    elif arch == ModelTypes.DLV3:
        run_config.model_type = ModelTypes.DLV3
    run_config.lr = lr

    run = Run(run_config)
    runner.add_run(run)
    runner.train(
        run=run,
        trainer_config=trainer_config
    )
    test_result = runner.test(
        run=run,
        trainer_config=None
    )
    return test_result[0]['test_f1']

In [5]:
default_run_config = SdssRunConfig(
    model_config=SdssModelConfig(
        learning_rate=1e-4,
        batch_size=32,
        model_type=ModelTypes.UNET,
        loss_type=LossType.DICE
    ),
    augmentation=Augmentations.NONE,
    shuffle_train=True
)
trainer_config = OptunaTuneTrainerConfig(
    logger=runner.logger,
    max_epochs=10,
    timeout=60 * 60 * 2,
    direction="maximize",
    num_jobs=2
)

tuning_objective = partial(
    objective,
    runner=runner,
    default_run_config=default_run_config,
    trainer_config=trainer_config
)

In [6]:
from star_analysis.runner.runner import TuningModes

study = runner.tune(
    mode=TuningModes.PARALLEL,
    trainer_config=trainer_config,
    optuna_objective=tuning_objective,
)

[I 2023-06-27 12:21:07,874] A new study created in memory with name: no-name-6eca4396-4f50-43d6-8747-f4741e5e7ff4
[INFO][2023-06-27 12:21:08,241] image_downloader.py:download_exact:62: Downloading 8162 6 0080
[INFO][2023-06-27 12:21:08,251] image_downloader.py:download_exact:62: Downloading 8162 6 0080


"batch_size":    32
"image_shape":   (224, 224)
"learning_rate": 0.0001
"loss_mode":     multilabel
"loss_module":   None
"loss_type":     DICE
"model_module":  None
"model_type":    UNET
"num_classes":   2
"batch_size":    32
"image_shape":   (224, 224)
"learning_rate": 0.0001
"loss_mode":     multilabel
"loss_module":   None
"loss_type":     DICE
"model_module":  None
"model_type":    UNET
"num_classes":   2


[INFO][2023-06-27 12:21:26,880] image_downloader.py:download_exact:62: Downloading 8162 6 0080
[INFO][2023-06-27 12:21:26,978] image_downloader.py:download_exact:62: Downloading 8162 6 0080
INFO: GPU available: True (mps), used: True
[INFO][2023-06-27 12:21:44,926] rank_zero.py:_info:48: GPU available: True (mps), used: True
INFO: TPU available: False, using: 0 TPU cores
[INFO][2023-06-27 12:21:44,927] rank_zero.py:_info:48: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
[INFO][2023-06-27 12:21:44,928] rank_zero.py:_info:48: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
[INFO][2023-06-27 12:21:44,928] rank_zero.py:_info:48: HPU available: False, using: 0 HPUs
[INFO][2023-06-27 12:21:45,018] image_downloader.py:download:46: Downloading 8162
INFO: GPU available: True (mps), used: True
[INFO][2023-06-27 12:21:45,026] rank_zero.py:_info:48: GPU available: True (mps), used: True
INFO: TPU available: False, using: 0 TPU core

Found 947 data/label pairs
Found 1 data/label pairs


INFO: 
  | Name         | Type     | Params
------------------------------------------
0 | loss_fn      | DiceLoss | 0     
1 | architecture | Unet     | 14.3 M
------------------------------------------
14.3 M    Trainable params
0         Non-trainable params
14.3 M    Total params
57.339    Total estimated model params size (MB)
[INFO][2023-06-27 12:22:06,055] model_summary.py:summarize:82: 
  | Name         | Type     | Params
------------------------------------------
0 | loss_fn      | DiceLoss | 0     
1 | architecture | Unet     | 14.3 M
------------------------------------------
14.3 M    Trainable params
0         Non-trainable params
14.3 M    Total params
57.339    Total estimated model params size (MB)


...

Training: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
study.best_params